In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
book_List = pd.read_csv("C:/프로젝트/k-d/2-Project/주제/data/998.final_data_set/final_data_set.csv")

# ISBN_THIRTEEN_NO 열을 문자열로 변환
book_List['ISBN_THIRTEEN_NO'] = book_List['ISBN_THIRTEEN_NO'].astype(str)

book_List.head(6)

,ISBN_THIRTEEN_NO,RANK_CO,KDC_NM,LON_CO,AGE_FLAG_NM,SEXDSTN_FLAG_NM,Yes24_Price,Yes24_Score,Yes24_Review,KDC_Category,BOOK_INTRCN_CN_EMBEDED,BOOK_TITLE_NM_EMBEDED
0,9788956593838,849.0,327.856,140.0,6.0,0.0,-1.0,9.27,69.0,2,[-6.48469925e-02 5.62208220e-02 1.19394280e-...,[ 0.15565178 -0.0734394 0.0099606 0.027090...
1,9788956993362,608.0,593.500,26.0,4.0,0.0,22500.0,9.64,65.0,0,[ 4.69394028e-02 -9.50393006e-02 4.61710058e-...,[-4.95723002e-02 -1.50724128e-01 -5.16865999e-...
2,9791162540466,239.0,327.856,43.0,4.0,0.0,16200.0,9.28,192.0,2,[-1.60625614e-02 -3.18203643e-02 4.19233739e-...,[ 1.11497588e-01 -4.69458699e-02 -2.94633303e-...
3,9791185585826,239.0,332.600,43.0,4.0,0.0,15300.0,9.34,65.0,2,[ 0.00990802 0.04011224 0.01613949 0.110284...,[ 1.69968084e-01 -4.10004193e-03 -2.14566644e-...
4,9791185952987,239.0,189.200,43.0,4.0,0.0,13500.0,9.40,815.0,8,[-1.93825993e-03 -4.79751937e-02 7.79000670e-...,[ 0.1210877 -0.08673159 0.03957615 0.025937...
5,9791187444084,239.0,325.211,43.0,4.0,0.0,12420.0,9.13,46.0,2,[-0.01703039 0.01445596 0.04210239 0.038949...,[ 0.09397484 -0.03083867 0.01408118 0.020640...


In [3]:
book_List.shape

(56305, 12)

In [4]:
book_List.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56305 entries, 0 to 56304
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ISBN_THIRTEEN_NO        56305 non-null  object 
 1   RANK_CO                 56305 non-null  float64
 2   KDC_NM                  56305 non-null  float64
 3   LON_CO                  56305 non-null  float64
 4   AGE_FLAG_NM             56305 non-null  float64
 5   SEXDSTN_FLAG_NM         56305 non-null  float64
 6   Yes24_Price             56305 non-null  float64
 7   Yes24_Score             56305 non-null  float64
 8   Yes24_Review            56305 non-null  float64
 9   KDC_Category            56305 non-null  int64  
 10  BOOK_INTRCN_CN_EMBEDED  56305 non-null  object 
 11  BOOK_TITLE_NM_EMBEDED   56305 non-null  object 
dtypes: float64(8), int64(1), object(3)
memory usage: 5.2+ MB


In [5]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 개발자가 설정한 중요도에 따른 가중치
weight_kdc_nm = 0.3  # 장르의 중요도
weight_score = 0.2  # 점수의 중요도
weight_price = 0.2  # 가격의 중요도
weight_review = 0.3  # 리뷰의 중요도

# 정상적인 ISBN_THIRTEEN_NO를 입력하기 위해 사용자에게 입력 받음
user_input_isbn = input("올바른 형식의 ISBN_THIRTEEN_NO를 입력하세요: ")

# 입력한 ISBN에 해당하는 행 인덱스 찾기
row_index = book_List[book_List['ISBN_THIRTEEN_NO'] == user_input_isbn].index

if not row_index.empty:
    # 'BOOK_INTRCN_CN_EMBEDED', 'KDC_NM', 'Yes24_Score', 'Yes24_Price', 'Yes24_Review' 열의 값을 리스트로 변환
    vector1_list = np.fromstring(str(book_List.at[row_index[0], 'BOOK_INTRCN_CN_EMBEDED']).replace('[', '').replace(']', ''), sep=' ')
    kdc_nm = book_List.at[row_index[0], 'KDC_NM']
    score = book_List.at[row_index[0], 'Yes24_Score']
    price = book_List.at[row_index[0], 'Yes24_Price']
    review = book_List.at[row_index[0], 'Yes24_Review']

    # 각 항목에 가중치를 부여하여 점수 계산
    score_kdc_nm = vector1_list[-1] * weight_kdc_nm
    score_score = score * weight_score 
    score_price = price * weight_price # 가격이 낮을수록 추천
    score_review = review * weight_review

    # 전체 점수 계산
    total_score = score_kdc_nm + score_score + score_price + score_review

    # 리스트를 합쳐서 하나의 벡터로 만듦
    vector1 = np.concatenate((vector1_list, [kdc_nm, score, price, review, total_score]), axis=None)[:-1]

    # 다른 책의 벡터를 가져와서 비교
    similarity_list = []
    for i in range(len(book_List)):
        # Check if the values are not float
        if not isinstance(book_List.at[i, 'BOOK_INTRCN_CN_EMBEDED'], float):
            vector2_list = np.fromstring(str(book_List.at[i, 'BOOK_INTRCN_CN_EMBEDED']).replace('[', '').replace(']', ''), sep=' ')
            vector2 = np.concatenate((vector2_list, [book_List.at[i, 'KDC_NM'], book_List.at[i, 'Yes24_Score'],
                                                    book_List.at[i, 'Yes24_Price'], book_List.at[i, 'Yes24_Review']]), axis=None)

            # 각 항목에 가중치를 부여하여 점수 계산
            score_kdc_nm = vector2_list[-1] * weight_kdc_nm
            score_score = book_List.at[i, 'Yes24_Score'] * weight_score
            score_price = book_List.at[i, 'Yes24_Price'] * weight_price
            score_review = book_List.at[i, 'Yes24_Review'] * weight_review

            # 전체 점수 계산
            total_score = score_kdc_nm + score_score + score_price + score_review

            # 코사인 유사도 계산
            similarity = cosine_similarity([vector1[:-1]], [vector2[:-1]])

            similarity_list.append((book_List.at[i, 'ISBN_THIRTEEN_NO'], i, similarity[0][0], total_score))

    # 상위 10개의 유사도 및 각 책의 점수 출력
    top_10_similarities = sorted(similarity_list, key=lambda x: (x[2], x[3]), reverse=True)[:10]
    for rank, (isbn, book_index, sim, total_score) in enumerate(top_10_similarities, start=1):
        # print(f"Top {rank}: Similarity with book {book_index} (ISBN: {isbn}): {sim}, Total Score: {total_score}")
        print(f"Top {rank}: ISBN: {isbn}, Price: {book_List.at[book_index, 'Yes24_Price']}, Score: {book_List.at[book_index, 'Yes24_Score']}, Review: {book_List.at[book_index, 'Yes24_Review']}, Genre Score: {book_List.at[book_index, 'KDC_NM']}, Similarity: {sim}")
        
else:
    print("입력한 ISBN에 해당하는 책이 데이터에 없습니다.")

# 테스트 데이터 = 9788901110691,   나는 세계일주로 경제를 배웠다, 	코너우드먼 지음 ;홍선영 옮김


Top 1: ISBN: 9788901110691, Price: 16200.0, Score: 8.54, Review: 81.0, Genre Score: 320.04, Similarity: 1.0
Top 2: ISBN: 9788998602260, Price: 16200.0, Score: 8.79, Review: 46.0, Genre Score: 320.04, Similarity: 0.9999999962504341
Top 3: ISBN: 9791189344351, Price: 16200.0, Score: 9.17, Review: 57.0, Genre Score: 320.911, Similarity: 0.9999999949981841
Top 4: ISBN: 9791159253614, Price: 19800.0, Score: 10.0, Review: 2.0, Genre Score: 392.19, Similarity: 0.999999994531169
Top 5: ISBN: 9791189344207, Price: 16650.0, Score: 9.04, Review: 64.0, Genre Score: 327.87, Similarity: 0.9999999943190327
Top 6: ISBN: 9791196073145, Price: 16650.0, Score: 9.4, Review: 53.0, Genre Score: 327.87, Similarity: 0.9999999937230648
Top 7: ISBN: 9788959896837, Price: 16200.0, Score: 9.52, Review: 582.0, Genre Score: 320.911, Similarity: 0.9999999935597419
Top 8: ISBN: 9791163430162, Price: 16650.0, Score: 9.18, Review: 63.0, Genre Score: 327.856, Similarity: 0.9999999935570656
Top 9: ISBN: 9788934973232, Pr

In [6]:
import csv
import os
import requests
from bs4 import BeautifulSoup

headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

Book_count = 0

for rank, (isbn, book_index, sim, total_score) in enumerate(top_10_similarities, start=1):
   response = requests.get("https://www.yes24.com/Product/Search?domain=ALL&query={0}".format(isbn), headers=headers)
   html = ("https://www.yes24.com/Product/Search?domain=ALL&query={0}".format(isbn))

   print(rank,"번 : ", html)


   html = response.content.decode('utf-8','replace')
   soup = BeautifulSoup(html, 'html.parser',from_encoding='cp949')

   my_st = soup.select(
      '#yesSchList > li'
   )

   for stock_name in my_st:
      texts = [element.text for element in soup.select("#yesSchList > li > div > div.item_info > div.info_row.info_name > a.gd_name")]
      print(texts)
      print("===================================================")
            

1 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9788901110691
['나는 세계일주로 경제를 배웠다']


c:\Users\516\AppData\Local\Programs\Python\Python38\lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


2 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9788998602260
['유쾌한 이코노미스트의 스마트한 경제 공부']
3 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9791189344351
['나는 오늘부터 경제기사를 읽기로 했다']
4 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9791159253614
['세상을 뒤흔든 전투의 역사']
5 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9791189344207
['오윤섭의 부동산 가치투자']
6 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9791196073145
['경매야 부탁해']
7 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9788959896837
['트렌드 코리아 2021']
8 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9791163430162
['주식 고수들만 아는 애널리스트 리포트 200% 활용법']
9 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9788934973232
['무엇이 아름다움을 강요하는가']
10 번 :  https://www.yes24.com/Product/Search?domain=ALL&query=9788973433414
['환율전쟁']
